# BBO-Rietveld テンプレート（YOUR_MATERIAL）

このノートブックは、ご自身の材料データに対してブラックボックス最適化（Optuna）でGSAS-IIのリートベルト精密化を自動化するためのテンプレートです。

- まず「設定」セルで STUDY_NAME や入出力ディレクトリ、初期プロジェクト（.gpx）などを調整してください。
- 次に、objective 関数内に探索空間と精密化の手順を実装してください（Y2O3/DSMO/LiCoO2 のノートを参考）。
- 実行は上から順にセルを実行します。

参考ドキュメント
- JupyterLab: https://jupyterlab.readthedocs.io/en/stable/
- GSASIIscriptable: https://gsas-ii.readthedocs.io/en/latest/GSASIIscriptable.html
- Optuna: https://optuna.readthedocs.io/en/stable/


In [ ]:
# 必要パッケージの読み込み
%matplotlib inline

import os
import sys
from multiprocessing import Process, Queue
import pandas as pd
import optuna
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# 注意: GSAS-II は環境にインストール済みのため、sys.path.append は不要です
# 使う箇所（クラスや関数内）で `import GSASIIscriptable as G2sc` を行います


In [ ]:
# 設定セル（まずここをあなたの材料に合わせて編集）
# STUDY_NAME: 解析対象の材料名（出力フォルダ名にも使用）
# RANDOM_SEED: Optuna の乱数シード（再現性確保）
# DATA_DIR: 入力データ (初期 gpx, PRM, CIF, CSV 等) を置いたフォルダ
# WORK_DIR: 最適化過程で生成される *.gpx や履歴DB等の出力先

STUDY_NAME = 'NaCl'
RANDOM_SEED = 1024

DATA_DIR = 'data/' + STUDY_NAME
WORK_DIR = 'work/' + STUDY_NAME + '2'


In [ ]:
# [新規セル: 3番目のセルの後に追加]
# Y2O3コード(アプローチ2)の前提(config)を準備
config = {
    'STUDY_NAME': STUDY_NAME,
    'DATA_DIR': DATA_DIR,
    'WORK_DIR': WORK_DIR,
    'RANDOM_SEED': RANDOM_SEED,
    # Y2O3のProjectクラスが他に必要な設定があればここに追加
}

In [ ]:
# make directories
! rm -f $WORK_DIR/$STUDY_NAME*
! mkdir -p $WORK_DIR

In [ ]:
# [修正: 5番目のセル (ProjectBBO クラス)]

class ProjectBBO:
    # def __init__(self, trial_number): # 修正前
    def __init__(self, config, trial_number): # 修正後 (config を受け取る)
        import GSASIIscriptable as G2sc
        import shutil
        
        # configから設定を読み込む (グローバル変数をフォールバックとして使用)
        DATA_DIR_local = config.get('DATA_DIR', DATA_DIR) 
        WORK_DIR_local = config.get('WORK_DIR', WORK_DIR)
        RANDOM_SEED_local = config.get('RANDOM_SEED', RANDOM_SEED)

        # Create a project with a default project name
        ### Change here (YOUR_PROJECT_FILE.gpx は NaCl の初期ファイル名に) ###
        gpx_seed_file = 'NaCl_init.gpx' # 例: 'NaCl_initial.gpx'
        seed_file_path = os.path.join(DATA_DIR_local, gpx_seed_file)
        gpx_trial_file = 'BBO_seed{0}_trial_{1}.gpx'.format(RANDOM_SEED_local, trial_number)
        trial_file_path = os.path.join(WORK_DIR_local, gpx_trial_file)

        try:
            shutil.copyfile(seed_file_path, trial_file_path)
        except FileNotFoundError:
             print(f"エラー: 初期GPXファイル {seed_file_path} が見つかりません。")
             print(f"ProjectBBOクラスの 'NaCl_init.gpx' を正しいファイル名に修正してください。")
             raise

        self.gpx = G2sc.G2Project(gpxfile=trial_file_path)

        # ... (以降の __init__ の中身は O3_NaCl.ipynb と同じ) ...
        self.hist1 = self.gpx.histograms()[0]
        self.phase0 = self.gpx.phases()[0]
        self.hist1.data['Instrument Parameters'][0]['I(L2)/I(L1)'] = [0.5, 0.5, 0]

        # Set to use iso (Uiso)
        for val in self.phase0.data['Atoms']:
            val[9] = 'I'

    def refine_and_calc_Rwp(self, param_dict):
        # このメソッドはY2O3のステップ実行と互換性があります (修正不要)
        self.gpx.do_refinements([param_dict])
        for hist in self.gpx.histograms():
            _, Rwp = hist.name, hist.get_wR()
        return Rwp

In [ ]:
# [修正: 6番目のセル (objective 関数)]

# Y2O3のコードで使われている sys をインポート
import sys 
# (os, shutil, G2sc などは ProjectBBO 内でインポートされる)

def objective(trial, config): # config を引数に取る
    """
    objective function for Optuna (Y2O3 アプローチベース)
    """
    
    ### define search space (Y2O3と同様) ###
    # Limits (acute angle)
    limits_lb = trial.suggest_float('Limits lower bound', 15, 130)
    limits_ub = trial.suggest_float('Limits upper bound', limits_lb + 20, 150)
    limits_refine = trial.suggest_categorical('limits refine', [True, False])
    refdict0 = {'set': {'Limits': [limits_lb, limits_ub]}, 'refine': limits_refine}

    # Background (Y2O3と同様)
    background_type = trial.suggest_categorical(
        'Background type', ['chebyschev',
                            'cosine',
                            'Q^2 power series',
                            'Q^-2 power series',
                            'lin interpolate',
                            'inv interpolate',
                            'log interpolate'])
    no_coeffs = trial.suggest_int('Number of coefficients', 1, 15)
    background_refine = trial.suggest_categorical('Background refine', [True, False])
    refdict0bg_h = {
        'set': {
            'Background': {
                'type': background_type,
                'no. coeffs': no_coeffs,
                'refine': background_refine
            }
        }
    }

    # Instrument parameters (Y2O3と同様)
    instrument_parameters1_refine = []
    for p in ['Zero']: # ゼロ点シフト
        if trial.suggest_categorical('Instrument_parameters refine %s' % (p), [True, False]):
            instrument_parameters1_refine.append(p)
    # 格子定数(Cell)も常に精密化対象に (Y2O3のコードに準拠)
    refdict1_h = {'set': {'Cell': True, 'Instrument Parameters': instrument_parameters1_refine}}

    # Sample parameters (Y2O3と同様)
    sample_parameters1_refine =[]
    for p in ['DisplaceX', 'DisplaceY', 'Scale']: # スケール因子など
        if trial.suggest_categorical('Sample_parameters refine %s' % (p), [True, False]):
            sample_parameters1_refine.append(p)
    refdict1_h2 = {"set": {'Sample Parameters': sample_parameters1_refine }}

    # Peakshape parameters (Y2O3と同様)
    instrument_parameters2_refine = []
    for p in ['U', 'V', 'W', 'X', 'Y', 'SH/L']: # プロファイル関数
        if trial.suggest_categorical('Peakshape_parameters refine %s' % (p), [True, False]):
            instrument_parameters2_refine.append(p)
    refdict2_h = {'set': {'Instrument Parameters': instrument_parameters2_refine}}

    # --- (A) NaCl用に修正 (原子パラメータ) ---
    # Y2O3 (refdict3_h = {'set': {'Atoms': {'all': 'XU'}}}) 
    # NaCl (Fm-3m) では X (座標) は精密化不要。U (Uiso) のみ。
    # ここではY2O3の 'XU' ではなく 'U' を指定します。
    refdict3_h = {'set': {'Atoms': {'all': 'U'}}}

    # Limits (wide angle) (Y2O3と同様)
    refdict_fin_h = {'set': {'Limits': [15, 150]}, 'refine': True}

    # Evaluate
    refine_params_list = [refdict0,
                          refdict0bg_h,
                          refdict1_h,
                          refdict1_h2,
                          refdict2_h,
                          refdict3_h,
                          refdict_fin_h]
    
    def evaluate(config_local, trial_number_local, refine_params_list_local):
        ERROR_PENALTY = 1e9      
        try:
            # O3_NaCl.ipynb の ProjectBBO クラスを呼び出す
            project = ProjectBBO(config_local, trial_number_local) 
            
            Rwp = None # Rwpを初期化
            for params in refine_params_list_local:
                Rwp = project.refine_and_calc_Rwp(params)
            
            if Rwp is None: # 精密化が一度も行われなかった場合
                return ERROR_PENALTY

            # --- (B) NaCl用に修正 (Uisoの検証) ---
            # (ProjectBBOの__init__でUisoが有効化されている前提)
            phase_NaCl = project.gpx.phases()[0]
            
            # Y2O3の .atoms() メソッドの代わりに、ProjectBBO と
            # 一貫性のあるデータ辞書 (data['Atoms']) にアクセスします。
            # (Uisoが10番目の要素であると仮定)
            try:
                u_iso_list = [atom[10] for atom in phase_NaCl.data['Atoms']]
                if min(u_iso_list) < 0:
                    # Uiso < 0 (物理的にあり得ない)
                    Rwp = ERROR_PENALTY
            except (IndexError, TypeError):
                 print(f"Trial {trial_number_local}: Uisoの検証に失敗。Atomsデータ構造を確認してください。", file=sys.stderr)
                 # Uisoが取得できない場合もペナルティ
                 Rwp = ERROR_PENALTY
                 
            return Rwp
            
        except Exception as e:
            # Refinement failed
            print(f"Trial {trial_number_local} は評価中に失敗しました: {e}", file=sys.stderr)
            return ERROR_PENALTY
            
    Rwp = evaluate(config, trial.number, refine_params_list)
    
    return Rwp

In [ ]:
# Optuna スタディの作成（履歴は WORK_DIR のSQLiteに保存）
study = optuna.create_study(
    study_name=f"{STUDY_NAME}_seed{RANDOM_SEED}",
    storage=f"sqlite:///{WORK_DIR}/history_sqlite.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(n_startup_trials=20, seed=RANDOM_SEED),
)


最適化を実行する前に objective 関数を実装してください。試行回数 (n_trials) はデータの難易度と計算時間に応じて調整します。例: 50, 100, 200 など。


In [ ]:
# [修正: 9番目のセル (study.optimize)]

# (元のコメントアウト) # study.optimize(objective, n_trials=100)

# config辞書が定義されているか確認
if 'config' not in globals():
    print("エラー: 'config' 辞書が定義されていません。(新規セルを3番目のセルの後に追加してください)")
else:
    print("アプローチ2 (Y2O3方式) で最適化を開始します...")
    # ラムダ式 (lambda) を使って config を objective 関数に渡す
    study.optimize(lambda trial: objective(trial, config=config), n_trials=100)

In [ ]:
# 結果の整形（Rwp最小の試行が上に来るようソート）
df = study.trials_dataframe()
df.columns = [''.join(col).replace('params', '').strip() for col in df.columns.values]
df.rename(columns={'value': 'Rwp', 'number': 'trial'}, inplace=True)
df.sort_values('Rwp')


In [ ]:
# Best configuration
study.best_params

In [ ]:
# Best Rwp
study.best_value

In [ ]:
# 成果物の保存（任意）
import json

# 全試行履歴をCSVとして保存
df.to_csv(f"{WORK_DIR}/trials.csv", index=False)

# 最良試行のパラメータをJSON保存
with open(f"{WORK_DIR}/best_params.json", "w") as f:
    json.dump(study.best_params, f, indent=2)

# 参考: 最良試行のGPXからCIFを書き出す（必要に応じて有効化）
import GSASIIscriptable as G2sc
gpx = G2sc.G2Project(f"{WORK_DIR}/{STUDY_NAME}_seed{RANDOM_SEED}_trial_{study.best_trial.number}.gpx")
phase = gpx.phases()[0]
phase.export_CIF(f"{WORK_DIR}/{STUDY_NAME}_best.cif", quickmode=True)

In [ ]:
# Rwp plot
def rwp_plot():
    minvalues = [df.iloc[0]['Rwp']]
    for i in range(1, df.shape[0]):
        minvalues.append(min(minvalues[-1], df.iloc[i]['Rwp']))
    minvalues = pd.DataFrame(minvalues)
    
    minvalues.plot(legend=None)
#     plt.ylim([6, 16])
    plt.grid(color='#cccccc')
    plt.ylabel('$R_{wp}$')
    plt.xlabel('Number of trials')
    plt.show()
    
rwp_plot()

In [ ]:
# Rietveld plot
def rietveld_plot():
    import GSASIIscriptable as G2sc

    gpx = G2sc.G2Project(
        '%s/%s_seed%s_trial_%s.gpx' % (WORK_DIR, STUDY_NAME, RANDOM_SEED, study.best_trial.number))

    hist1 = gpx.histograms()[0]
    phase0 = gpx.phases()[0]

    hist = hist1
    i = 5
    two_theta = hist.getdata("X")[::i]
    Yobs = hist.getdata("Yobs")[::i]
    Ycalc = hist.getdata("Ycalc")[::i]
    bg = hist.getdata("Background")[::i]
    residual = hist.getdata("Residual")[::i]

    fig = plt.figure()
    gs = GridSpec(5, 1, figure=fig)
    ax1 = fig.add_subplot(gs[:4, :])
    ax2 = fig.add_subplot(gs[4, :])
    fig.subplots_adjust(hspace=0)
    ax1.grid(color='#cccccc')

    ax1.scatter(two_theta, Yobs, marker='P', lw=0.0001, c='Black', label='XRD (Obs)')
    ax1.plot(two_theta, Ycalc, label='XRD (Calc)')
    ax1.plot(two_theta, bg, color='red', label='Background (Calc)')
    ax1.set_ylabel('Intensity')
    ax1.legend()
    ax2.plot(two_theta, residual, color='blue')
    plt.setp(ax1.get_xticklabels(), visible=False);
    # ax2.set_ylim(-6600, 6600)
    plt.xlabel(r'$2\theta$ (deg.)')
    ax2.set_ylabel('Residual')
    # change 2theta range according to your data
    ax1.set_xlim(15, 150)
    ax2.set_xlim(15, 150)
    plt.show()
    
rietveld_plot()